# Keras入门课5：网络可视化及训练监控

本节专注于Keras中神经网络的可视化，包括网络结构可视化以及如何使用TensorBoard来监控训练过程。  
这里我们借用第2课的代码内容来进行示例和讲解。

网络前面的定义、数据初始化都一样，主要是fit函数

#### 启用TensorBoard
在model的fit函数中加入TensorBoard的回调函数即可，训练数据就会自动保存在log_dir指定的目录内，然后在命令行启动命令 tensorboard --logdir=./log 即可。TensorBoard会记录loss及model.metrics里面的值，本例中即acc,loss,val_acc,val_loss四个值，每个epoch更新一次。  
除了这些SCALARS，还会记录网络的GRAPH，直接可视化网络结构，但是相比用原生TensorFlow生成的图而言，相差还是比较大的，比较难看，所以不推荐在Keras中使用TensorBoard查看网络结构。

我只训练了2个epoch，所以只记录下了两个值。曲线图如下
![](./images/tensorboard_scalars.png)

↓直方图，用来统计参数的分布

![](./images/tensorboard_hist.png)

In[1]:

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
# 引入Tensorboard
from keras.callbacks import TensorBoard
from keras.utils import plot_model

# (x_train,y_train),(x_test,y_test) = mnist.load_data() # out: np.ndarray

# x_train = x_train.reshape(-1,28,28,1)
# x_test = x_test.reshape(-1,28,28,1)
# input_shape = (28,28,1)

# y_train = keras.utils.to_categorical(y_train,10)
# y_test = keras.utils.to_categorical(y_test,10)

# x_train = x_train/255
# x_test = x_test/255

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
 
x_train, y_train = mnist.train.images, mnist.train.labels
x_test, y_test = mnist.test.images, mnist.test.labels


x_train = x_train/255
x_test = x_test/255


Using TensorFlow backend.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [10]:
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)
input_shape = (28,28,1)

In [4]:


model = Sequential()
model.add(Conv2D(filters = 32,kernel_size=(3,3),
                 activation='relu',input_shape = input_shape,name='conv1'))
model.add(Conv2D(64,(3,3),activation='relu',name='conv2'))
model.add(MaxPooling2D(pool_size=(2,2),name='pool2'))
model.add(Dropout(0.25,name='dropout1'))
model.add(Flatten(name='flat1'))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5,name='dropout2'))
model.add(Dense(10,activation='softmax',name='output'))

In [11]:
# plot_model(model,to_file='model.png')

↑keras的utils里面专门有一个plot_model函数是用来可视化网络结构的，为了保证格式美观，我们在定义模型的时候给每个层都加了一个名字。   
对于大多数的Keras的layers，都有name这一参数。
使用plot_model就可以生成类似下图的一张图片，相比TensorBoard的Graph要清晰明了很多。所以在Keras中打印图结构还是推荐使用Keras自带的方法。

![](./images/mlp_model.png)

In [12]:
model.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics=['accuracy'])

TensorBoard接口函数，有很多参数可选，具体细节可以参看官方文档。相比TensorFlow中的summary保存而言，keras中的TensorBoard使用更为简单，但是灵活性较差，只适合一些最基础的使用。

In [7]:
!mkdir logs

In [8]:
tb = TensorBoard(log_dir='./logs',  # log 目录
                 histogram_freq=1,  # 按照何等频率（epoch）来计算直方图，0为不计算
                 batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=False, # 是否可视化梯度直方图
                 write_images=False,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)
callbacks = [tb]

In [13]:
model.fit(x_train,y_train,batch_size=64,epochs=2
          ,verbose=1,validation_data=(x_test,y_test),
          callbacks=callbacks)

Train on 55000 samples, validate on 10000 samples
Epoch 1/2
55000/55000 [==============================] - 171s 3ms/step - loss: 2.3017 - acc: 0.1118 - val_loss: 2.3012 - val_acc: 0.1135
Epoch 2/2
55000/55000 [==============================] - 212s 4ms/step - loss: 2.3015 - acc: 0.1123 - val_loss: 2.3011 - val_acc: 0.1135


## 总结

1. 学习了如何用TensorBoard监控训练过程
1. 学习了如何使用keras自带的save_model函数来保存网络图


在使用plot_model绘制图片的时候还遇到了一些错误，如果你也报错，可以参看我的另一篇文章尝试解决：[ Mac下使用Keras plot_model函数时出错的解决办法](http://blog.csdn.net/tsyccnh/article/details/78866976)


参考：
> https://keras.io/visualization/
